In [1]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 CUDAFLAGS="-arch=all -lcublas" pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir --verbose

In [ ]:
#cpu
#%env CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"
#cuda
#%env CMAKE_ARGS="-DLLAMA_CUBLAS=on" 
#%env FORCE_CMAKE=1
#%env CUDAFLAGS="-arch=all -lcublas"
#%pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir --verbose

In [2]:
%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.


In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [ ]:
#from huggingface_hub import hf_hub_download
#hf_hub_download(repo_id='TheBloke/Mistral-7B-v0.1-GGUF', filename='mistral-7b-v0.1.Q4_K_M.gguf', local_dir='./models', local_dir_use_symlinks=False)

In [4]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
model_url = "https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/resolve/main/mistral-7b-v0.1.Q4_K_M.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=None,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path='models/mistral-7b-v0.1.Q4_K_M.gguf',
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 0},
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral-7b-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_

In [5]:
%pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import psycopg2

# db_name = "vector_db"
# host = "localhost"
# password = "postgres"
# port = "5432"
# user = "postgres"
# # conn = psycopg2.connect(connection_string)
# conn = psycopg2.connect(
#     dbname=postgres,
#     host=host,
#     password=password,
#     port=port,
#     user=user,
# )
# conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"DROP DATABASE IF EXISTS {db_name}")
#     c.execute(f"CREATE DATABASE {db_name}")


In [6]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

db_name = "vectordb"
host = "db"
password = "testpwd"
port = "5432"
user = "testuser"
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

In [ ]:
#%mkdir data
#%wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

In [7]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

In [8]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))



In [9]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [10]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding


In [11]:
vector_store.add(nodes)

['27cfeb77-43f3-4a3b-aad5-3ecc0c8d3e96',
 '65632974-12ba-4fc1-9e43-90edbdd9f0e3',
 '2c03df25-9a0d-41f1-9687-6b4c8d495958',
 'f16c1867-6d30-409c-a678-cb7b617e0ae4',
 '9eec1132-3943-4117-8ef2-385457f62ec8',
 '9f90eec8-2c52-4c96-bcc6-d89c4b66af70',
 '8677b1ff-d292-4fa1-8f8d-f3da70a3d054',
 '6f6dfa27-3244-4fa0-98ff-c72b2497b050',
 '25947634-f4f6-4a9a-bd82-698da32b212b',
 'fb4b009f-0832-466b-a710-09a08dd5c517',
 'f22046f8-f366-4d10-beac-647c17ead68c',
 'a94df219-d8ac-4a4d-87b6-8c88d604484a',
 '254b049f-37aa-4518-88f7-05aacc7a5a82',
 'a39401e5-70eb-4901-a488-0db1ba297b4e',
 'bede8efd-f8ad-427a-88c1-24b2624632ab',
 'eb9175e5-127f-4e58-9bb9-b205fc1ebecb',
 '8efeab17-3f9e-4e44-be68-521e1b37151c',
 'f4c9e034-499a-4c81-bfaf-1bc0cd1109a9',
 'd725d0c9-1e66-4194-9a49-59d293a9a1ef',
 '78c3a840-704f-4245-8b6c-b2085b6086a4',
 '23289599-d342-4180-829d-f7b7bea21c7c',
 '0954c3c1-5d18-4b03-88ae-a8d69855cb34',
 '39702a30-f11b-4e67-a9b3-d283b2a4bb1b',
 'f89e5c24-675e-479a-a5e2-b06bfd8cb964',
 'e1dac721-b709-

In [12]:
query_str = "Can you tell me about the key concepts for safety finetuning"

In [13]:
query_embedding = embed_model.get_query_embedding(query_str)

In [14]:
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [15]:
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())


TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

In [16]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))


In [17]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [18]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [19]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm, streaming=True)

In [20]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)
response.print_response_stream()

1. Llama 2 performs better than other open-source models on most benchmarks. 2. Llama 2 performs better than other open-source models on most benchmarks. 3. Llama 2 performs better than other open-source models on most benchmarks. 4. Llama 2 performs better than other open-source models on most benchmarks. 5. Llama 2 performs better than other open-source models on most benchmarks.


llama_print_timings:        load time =   62833.78 ms
llama_print_timings:      sample time =      66.00 ms /   105 runs   (    0.63 ms per token,  1590.93 tokens per second)
llama_print_timings: prompt eval time =   92090.75 ms /   764 tokens (  120.54 ms per token,     8.30 tokens per second)
llama_print_timings:        eval time =   42160.42 ms /   104 runs   (  405.39 ms per token,     2.47 tokens per second)
llama_print_timings:       total time =  134971.48 ms /   868 tokens


In [ ]:
print(str(response))


In [ ]:
print(response.source_nodes[0].get_content())


In [ ]:
query_str = "What is the parameter count of a Llama 2 model?"
response = query_engine.query(query_str)
response.print_response_stream()

In [ ]:
print(str(response))
